# 07 - Autonomous Workflows

**Build agents that plan and execute complex tasks autonomously!**

## Learning Objectives

By the end of this notebook, you will:
- Implement planning and task decomposition
- Build self-correction mechanisms
- Design human-in-the-loop patterns
- Add safety guardrails

## Table of Contents

1. [Planning & Decomposition](#planning)
2. [Self-Correction](#self-correction)
3. [Human-in-the-Loop](#human-loop)
4. [Safety & Guardrails](#safety)
5. [Complete Workflow](#complete)
6. [Exercises](#exercises)
7. [Checkpoint](#checkpoint)

In [ ]:
# GUIDED: Setup
import os
import sys
import json
from pathlib import Path
from dataclasses import dataclass, field
from typing import Optional, Callable
from enum import Enum

sys.path.append(str(Path.cwd().parent))

from dotenv import load_dotenv
load_dotenv(Path.cwd().parent / ".env")

from openai import OpenAI
client = OpenAI()

print("Setup complete!")

---
## 1. Planning & Decomposition <a id='planning'></a>

Complex tasks need to be broken down into manageable steps.

In [ ]:
# GUIDED: Task planning prompt

PLANNING_PROMPT = """You are a task planning agent. Given a complex task, break it down into clear, actionable steps.

For each step, provide:
1. Step number
2. Description of what to do
3. Expected output
4. Dependencies (which previous steps must complete first)

Output as JSON:
{
  "goal": "The overall goal",
  "steps": [
    {
      "id": 1,
      "action": "What to do",
      "expected_output": "What we expect",
      "dependencies": []
    }
  ]
}
"""

def create_plan(task: str) -> dict:
    """Create an execution plan for a task."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": PLANNING_PROMPT},
            {"role": "user", "content": f"Plan this task: {task}"}
        ],
        response_format={"type": "json_object"}
    )
    
    return json.loads(response.choices[0].message.content)

# Test planning
task = "Build a simple web scraper that extracts article titles from a news website"
plan = create_plan(task)

print("Generated Plan:")
print(json.dumps(plan, indent=2))

In [ ]:
# GUIDED: Task execution with plan

@dataclass
class TaskStep:
    """A step in the execution plan."""
    id: int
    action: str
    expected_output: str
    dependencies: list[int] = field(default_factory=list)
    status: str = "pending"  # pending, running, completed, failed
    result: Optional[str] = None

class PlanExecutor:
    """Executes a plan step by step."""
    
    def __init__(self, plan: dict):
        self.goal = plan["goal"]
        self.steps = [
            TaskStep(
                id=s["id"],
                action=s["action"],
                expected_output=s["expected_output"],
                dependencies=s.get("dependencies", [])
            )
            for s in plan["steps"]
        ]
    
    def get_ready_steps(self) -> list[TaskStep]:
        """Get steps that are ready to execute."""
        ready = []
        for step in self.steps:
            if step.status == "pending":
                # Check if all dependencies are completed
                deps_complete = all(
                    self.get_step(d).status == "completed"
                    for d in step.dependencies
                )
                if deps_complete:
                    ready.append(step)
        return ready
    
    def get_step(self, step_id: int) -> TaskStep:
        """Get a step by ID."""
        for step in self.steps:
            if step.id == step_id:
                return step
        raise ValueError(f"Step {step_id} not found")
    
    def execute_step(self, step: TaskStep) -> str:
        """Execute a single step."""
        step.status = "running"
        
        # Get context from dependencies
        context = ""
        for dep_id in step.dependencies:
            dep = self.get_step(dep_id)
            context += f"\nStep {dep_id} result: {dep.result}"
        
        # Execute (simulated - would call actual tools/agents)
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Execute the task and provide the result."},
                {"role": "user", "content": f"Task: {step.action}\n{context}"}
            ]
        )
        
        result = response.choices[0].message.content
        step.result = result
        step.status = "completed"
        
        return result
    
    def run(self, verbose: bool = True) -> dict:
        """Execute all steps in order."""
        results = {}
        
        while True:
            ready = self.get_ready_steps()
            if not ready:
                break
            
            for step in ready:
                if verbose:
                    print(f"\nExecuting Step {step.id}: {step.action}")
                
                result = self.execute_step(step)
                results[step.id] = result
                
                if verbose:
                    print(f"  Result: {result[:100]}...")
        
        return results

# Execute the plan
executor = PlanExecutor(plan)
results = executor.run()

---
## 2. Self-Correction <a id='self-correction'></a>

Agents should be able to detect and fix their own mistakes.

In [ ]:
# GUIDED: Self-correction pattern

def execute_with_retry(
    task: str,
    validator: Callable[[str], tuple[bool, str]],
    max_attempts: int = 3
) -> str:
    """
    Execute a task with self-correction.
    
    validator: Function that returns (is_valid, feedback)
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Complete tasks carefully and fix any issues."},
        {"role": "user", "content": task}
    ]
    
    for attempt in range(max_attempts):
        print(f"\nAttempt {attempt + 1}:")
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        
        result = response.choices[0].message.content
        print(f"  Result: {result[:100]}...")
        
        # Validate the result
        is_valid, feedback = validator(result)
        
        if is_valid:
            print(f"  Status: Valid!")
            return result
        
        print(f"  Status: Invalid - {feedback}")
        
        # Add feedback for next attempt
        messages.append({"role": "assistant", "content": result})
        messages.append({"role": "user", "content": f"That's not quite right. {feedback} Please try again."})
    
    return result  # Return last attempt even if not valid

# Example validator
def validate_json(result: str) -> tuple[bool, str]:
    """Validate that result is valid JSON."""
    try:
        json.loads(result)
        return True, ""
    except json.JSONDecodeError as e:
        return False, f"Not valid JSON: {str(e)}"

# Test self-correction
result = execute_with_retry(
    "Create a JSON object with name, age, and occupation fields for a software developer",
    validate_json
)

In [ ]:
# GUIDED: Reflection pattern

def reflect_and_improve(task: str, initial_result: str) -> str:
    """
    Have the agent reflect on its work and improve it.
    """
    reflection_prompt = f"""You previously completed this task:
Task: {task}

Your output was:
{initial_result}

Please reflect on your work:
1. What did you do well?
2. What could be improved?
3. Are there any errors or omissions?

Then provide an improved version of your output."""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": reflection_prompt}]
    )
    
    return response.choices[0].message.content

# Test reflection
task = "Write a Python function that checks if a number is prime"
initial = """def is_prime(n):
    for i in range(2, n):
        if n % i == 0:
            return False
    return True"""

improved = reflect_and_improve(task, initial)
print("Reflection and Improvement:")
print(improved)

---
## 3. Human-in-the-Loop <a id='human-loop'></a>

Some decisions require human approval or input.

In [ ]:
# GUIDED: Human approval pattern

class ApprovalRequired(Exception):
    """Raised when human approval is needed."""
    def __init__(self, action: str, reason: str):
        self.action = action
        self.reason = reason
        super().__init__(f"Approval needed: {action}")

class HumanInLoopAgent:
    """Agent that requests human approval for certain actions."""
    
    def __init__(self, approval_callback: Callable[[str, str], bool] = None):
        self.requires_approval = [
            "delete", "remove", "modify", "update", "send", "execute"
        ]
        self.approval_callback = approval_callback or self._default_approval
    
    def _default_approval(self, action: str, details: str) -> bool:
        """Default approval - always prompts user (simulated)."""
        print(f"\n[APPROVAL REQUIRED]")
        print(f"Action: {action}")
        print(f"Details: {details}")
        # In real use, would prompt user
        # For demo, auto-approve
        return True
    
    def check_approval(self, action: str, details: str) -> bool:
        """Check if action requires and gets approval."""
        action_lower = action.lower()
        
        for trigger in self.requires_approval:
            if trigger in action_lower:
                return self.approval_callback(action, details)
        
        return True  # No approval needed
    
    def execute(self, action: str, details: str) -> str:
        """Execute an action with approval check."""
        if not self.check_approval(action, details):
            return "Action rejected by user"
        
        # Execute the action
        return f"Executed: {action}"

# Test human-in-loop
agent = HumanInLoopAgent()

print(agent.execute("Read file", "config.json"))  # No approval
print(agent.execute("Delete file", "old_data.csv"))  # Needs approval

---
## 4. Safety & Guardrails <a id='safety'></a>

Protect against unsafe or unintended actions.

In [ ]:
# GUIDED: Safety guardrails

class SafetyGuardrails:
    """Safety checks for agent actions."""
    
    def __init__(self):
        self.blocked_actions = [
            "rm -rf", "format", "sudo", "DROP TABLE", "DELETE FROM"
        ]
        self.rate_limits = {}  # action -> (count, timestamp)
        self.max_rate = 10  # max calls per minute
    
    def check_blocked(self, action: str) -> tuple[bool, str]:
        """Check if action is blocked."""
        action_lower = action.lower()
        for blocked in self.blocked_actions:
            if blocked.lower() in action_lower:
                return False, f"Blocked action: {blocked}"
        return True, ""
    
    def check_rate_limit(self, action_type: str) -> tuple[bool, str]:
        """Check rate limiting."""
        import time
        now = time.time()
        
        if action_type in self.rate_limits:
            count, start_time = self.rate_limits[action_type]
            if now - start_time < 60:  # Within 1 minute
                if count >= self.max_rate:
                    return False, f"Rate limit exceeded for {action_type}"
                self.rate_limits[action_type] = (count + 1, start_time)
            else:
                self.rate_limits[action_type] = (1, now)
        else:
            self.rate_limits[action_type] = (1, now)
        
        return True, ""
    
    def validate(self, action: str, action_type: str = "default") -> tuple[bool, str]:
        """Run all safety checks."""
        # Check blocked actions
        ok, msg = self.check_blocked(action)
        if not ok:
            return False, msg
        
        # Check rate limits
        ok, msg = self.check_rate_limit(action_type)
        if not ok:
            return False, msg
        
        return True, "Passed all checks"

# Test guardrails
guard = SafetyGuardrails()

print("Safe action:", guard.validate("read file config.json"))
print("Dangerous action:", guard.validate("rm -rf /"))
print("SQL injection:", guard.validate("DROP TABLE users"))

---
## 5. Complete Workflow <a id='complete'></a>

Let's put it all together into an autonomous workflow.

In [ ]:
# GUIDED: Complete autonomous workflow

class AutonomousWorkflow:
    """A complete autonomous workflow with planning, execution, and safety."""
    
    def __init__(self):
        self.guardrails = SafetyGuardrails()
        self.execution_log = []
    
    def plan(self, task: str) -> dict:
        """Create execution plan."""
        print("\n[PLANNING]")
        plan = create_plan(task)
        print(f"Created plan with {len(plan['steps'])} steps")
        return plan
    
    def execute_step(self, step: dict, context: str) -> tuple[bool, str]:
        """Execute a single step with safety checks."""
        action = step['action']
        
        # Safety check
        ok, msg = self.guardrails.validate(action)
        if not ok:
            return False, f"Blocked: {msg}"
        
        # Execute
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Execute the task."},
                {"role": "user", "content": f"{action}\n\nContext: {context}"}
            ]
        )
        
        result = response.choices[0].message.content
        return True, result
    
    def run(self, task: str, auto_approve: bool = True) -> dict:
        """Run the complete workflow."""
        results = {
            "task": task,
            "success": False,
            "steps": []
        }
        
        # Plan
        plan = self.plan(task)
        
        # Execute each step
        context = ""
        for step in plan['steps']:
            print(f"\n[EXECUTING] Step {step['id']}: {step['action']}")
            
            success, result = self.execute_step(step, context)
            
            step_result = {
                "id": step['id'],
                "action": step['action'],
                "success": success,
                "result": result[:200]
            }
            results['steps'].append(step_result)
            
            if success:
                context += f"\nStep {step['id']}: {result}"
                print(f"  Success: {result[:100]}...")
            else:
                print(f"  Failed: {result}")
                break
        
        results['success'] = all(s['success'] for s in results['steps'])
        return results

# Run workflow
workflow = AutonomousWorkflow()
result = workflow.run("Analyze the pros and cons of remote work")

print("\n" + "="*50)
print("Workflow Complete!")
print(f"Success: {result['success']}")
print(f"Steps completed: {len(result['steps'])}")

---
## 6. Exercises <a id='exercises'></a>

### Exercise 1: Add Retry Logic

Modify the workflow to retry failed steps.

In [ ]:
# TODO: Add retry logic to the workflow

# Your code here:


### Exercise 2: Progress Tracking

Add a progress tracker that shows completion percentage.

In [ ]:
# TODO: Add progress tracking

# Your code here:


### Exercise 3: Checkpoint/Resume

Implement saving progress and resuming from a checkpoint.

In [ ]:
# TODO: Add checkpoint/resume capability

# Your code here:


---
## 7. Checkpoint <a id='checkpoint'></a>

Before moving on, verify:

- [ ] You can implement task planning and decomposition
- [ ] You understand self-correction patterns
- [ ] You can add human-in-the-loop controls
- [ ] You implemented safety guardrails
- [ ] You completed at least 2 exercises

### Next Steps

In the final notebook, you'll build your own **Capstone Agent** - putting everything together!

---
## Summary

**Autonomous Workflow Components:**

| Component | Purpose |
|-----------|---------||
| Planning | Break tasks into steps |
| Execution | Run steps with context |
| Self-correction | Fix mistakes |
| Human-in-loop | Get approval |
| Guardrails | Prevent unsafe actions |

**Best Practices:**
- Always validate critical actions
- Implement rate limiting
- Log all actions
- Allow human override
- Set reasonable limits